In [1]:
import requests
import json
import pandas as pd
import csv
import time

##### Conexión a la API

In [3]:
EMAIL = ""
PASSWORD = ""
URL_LOGIN = "https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/"
headers = {"email": EMAIL,"password":PASSWORD}

In [4]:
response = requests.request("GET", URL_LOGIN, headers = headers)

In [5]:
response_json = json.loads(response.text)

In [6]:
accessToken = response_json["data"][0]["accessToken"]

In [7]:
headers_token = {"accessToken": accessToken}

## 3.1.1.1.1 Infoline(General)

In [8]:
dataref = "20200318"
URL_INFO_GRL = "https://openapi.emtmadrid.es/v2/transport/busemtmad/lines/info/" + dataref + "/"

In [9]:
response_info_grl = requests.request("GET", URL_INFO_GRL, headers = headers_token)

In [10]:
parsed = (json.loads(response_info_grl.text)["data"])
print(json.dumps(parsed, indent=4))

[
    {
        "line": "361",
        "label": "001",
        "nameA": "ATOCHA RENFE",
        "nameB": "MONCLOA",
        "group": "110",
        "startDate": "30/05/2020",
        "endDate": "30/05/2020",
        "longLine1": 4827,
        "longLine2": 4811,
        "order": 1,
        "color": "00aecf"
    },
    {
        "line": "362",
        "label": "002",
        "nameA": "PUERTA DE TOLEDO",
        "nameB": "ARG\u00dcELLES",
        "group": "110",
        "startDate": "30/05/2020",
        "endDate": "30/05/2020",
        "longLine1": 5276,
        "longLine2": 5415,
        "order": 1,
        "color": "00aecf"
    },
    {
        "line": "203",
        "label": "203",
        "nameA": "ATOCHA RENFE",
        "nameB": "AEROPUERTO",
        "group": "410",
        "startDate": "30/05/2020",
        "endDate": "30/05/2020",
        "longLine1": 20933,
        "longLine2": 22817,
        "order": 2,
        "color": "f1dd38",
        "forecolor": "050505"
    },
    {
      

### csv

In [11]:
def print_infoline_general(json):
    return [json["line"],json["label"],json["nameA"],json["nameB"],json["group"],
           json["longLine1"],json["longLine2"]]

In [18]:
res = []
for par in parsed:
    res.append(print_infoline_general(par))
pd.DataFrame(res,columns = ["line","label","nameA","nameB","group","longLine1","longLine2"]).to_csv('infoline.csv', sep=';', encoding='utf-8',index=False)

##### Número de lineas de autobús

In [93]:
lineas = pd.read_csv('infoline.csv', sep=";", header=0)

In [96]:
lineas.count()[0]

214

## 3.1.1.1.2 Line stops

##### Ejecución con una sola línea de autobús de ejemplo

In [19]:
line_id = "003"
direction = "from B to A"
URL_LINE_STOPS = "https://openapi.emtmadrid.es/v1/transport/busemtmad/lines/" + line_id + "/stops/" + direction

In [20]:
response_line_stops = requests.request("GET", URL_LINE_STOPS, headers = headers_token)

In [21]:
parsed = (json.loads(response_line_stops.text)["data"][0])
print(json.dumps(parsed, indent=4))

{
    "line": "003",
    "label": "3",
    "stops": [
        {
            "stop": "1885",
            "name": "Puerta de Toledo",
            "postalAddress": "Puerta de Toledo, 3",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -3.71146161659808,
                    40.4063523529995
                ]
            },
            "pmv": "61410",
            "dataLine": [
                "003"
            ]
        },
        {
            "stop": "1884",
            "name": "Puerta de Toledo",
            "postalAddress": "Gran V\u00eda de San Francisco, 2 con Gta. Puerta de Toledo",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -3.71212428707064,
                    40.4077627011361
                ]
            },
            "pmv": null,
            "dataLine": [
                "003",
                "060",
                "516"
            ]
        },

##### Obtención del conjunto de datos

In [2]:
def load_csv(path):
    with open(path,"r") as input_file:
        reader = csv.reader(input_file,delimiter = ";")
        matrix = [row for row in reader]
    return matrix

In [3]:
def print_csv(file_path, matrix,modificacion):
    with open(file_path, modificacion,newline='',encoding='utf-8') as output_file:
        writer = csv.writer(output_file, delimiter = ";")
        for row in matrix: 
            writer.writerow(row)

In [97]:
lineas = load_csv('infoline.csv')

In [98]:
def print_lines_stops_header(json,direction):
    header_a = json["stops"][0]
    header_b = json["stops"][-1]
    return[str(json["line"]) +  header_a["name"] + header_b["name"] , [json["line"],json["label"],direction,
            header_a["stop"],header_a["name"],header_a["postalAddress"],header_a["geometry"]["type"],
            header_a["geometry"]["coordinates"][0],header_a["geometry"]["coordinates"][1],
            header_b["stop"],header_b["name"],header_b["postalAddress"],header_b["geometry"]["type"],
            header_b["geometry"]["coordinates"][0],header_b["geometry"]["coordinates"][1]]]

In [ ]:
directions = ["From A to B","From B to A"]
res = []
aux = []
for linea in lineas:
    for direction in directions:
        try:
            URL_LINE_STOPS = "https://openapi.emtmadrid.es/v1/transport/busemtmad/lines/" + linea[0] + "/stops/" + direction + "/"
            response_line_stops = requests.request("GET", URL_LINE_STOPS, headers = headers_token)
            parsed = (json.loads(response_line_stops.text)["data"][0])
            result = print_lines_stops_header(parsed,direction)
            if(result[0] not in aux):
                res.append(print_lines_stops_header(parsed,direction)[1])
        except:
            print_csv('line_headers_info.csv',res,"a")
            res = []
            e1 = 0
print_csv('line_headers_info.csv',res,"a")

##### Comprobar si las cabeceras de las lineas son las mismas para ambas frecuencias

In [4]:
data = load_csv('line_headers_info.csv')

In [5]:
aux = []
res_new = []
for linea in data:
    repeat = linea[0],linea[3],linea[9]
    if repeat not in aux:
        aux.append(repeat)
        res_new.append(linea)
    print_csv('line_headers_info.csv',res_new,"w")

In [6]:
lineas = pd.read_csv('line_headers_info.csv', sep=";", header=None)

Obtenemos el mismo número de líneas, por los que las cabeceras son fijas para ambos sentidos

In [7]:
lineas.count()[0]

214

Limpiamos el conjunto de datos final

In [13]:
data = pd.read_csv('line_headers_info.csv', sep=";", header=None,encoding = "utf-8")

In [15]:
res = []
for key, linea in data.iterrows():
    res.append([linea[0],linea[1],"headerA",linea[3],linea[4],linea[8],linea[7]])
    res.append([linea[0],linea[1],"headerB",linea[9],linea[10],linea[14],linea[13]])
pd.DataFrame(res,columns = ["id_line","label","type_header","id_header","name_header","latitude","longitude"]).to_csv('line_headers_info.csv', sep=';', encoding='utf-8',index=False)